In [2]:
'Hello World!'

'Hello World!'

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

2.9.0+cu126
True
1
0
Tesla T4


In [4]:
import os
import math
import random
from glob import glob
import os
import kagglehub
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
os.environ["KAGGLE_USERNAME"] = "tejaskumarvurs"
os.environ["KAGGLE_KEY"] = "80fbc5d540819df3b4666ae5df969af9"

path = kagglehub.dataset_download("tejaskumarvurs/gen-ai-animal-dataset")

dest = "/content/animal_data"
if not os.path.exists(dest):
    shutil.copytree(path, dest)

print(f"✅ Dataset is ready at: {dest}")

100%|██████████| 8.91G/8.91G [01:39<00:00, 96.4MB/s]

Extracting files...


✅ Dataset is ready at: /content/animal_data


In [5]:
!find / -name "gen-ai-animal-dataset" -type d 2>/dev/null

/root/.cache/kagglehub/datasets/tejaskumarvurs/gen-ai-animal-dataset


In [6]:
# Force a link from the hidden system path to your visible content folder
!ln -s /root/.cache/kagglehub/datasets/tejaskumarvurs/gen-ai-animal-dataset/versions/1 /content/dataset

# Now check if Colab can 'see' into that shortcut
!ls /content/dataset | head -n 5

!ls /content/dataset/Camel | head -n 5

ln: failed to create symbolic link '/content/dataset/1': File exists
1
Bear
Brown bear
Bull
Butterfly
01d5030b1bb698d6.jpg
0215b972cb19e575.jpg
02db256a75c9419a.jpg
035205673c0ae617.jpg
049a26d67bd3192d.jpg


In [7]:
DATA_PATH = "/content/dataset"

def count_files(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

print(f"Total files accessible for T4 training: {count_files(DATA_PATH)}")

Total files accessible for T4 training: 29071


In [16]:
path = "/content/dataset"

if os.path.exists(path):
    contents = os.listdir(path)
    print(f"✅ Path exists!")
    print(f"Items inside '{path}': {contents[:5]}") # Shows first 5 items
    
    # Check if the first item is a file or a folder
    if len(contents) > 0:
        first_item = os.path.join(path, contents[0])
        if os.path.isdir(first_item):
            print("📁 Found subfolders (Classes). Use datasets.ImageFolder(path)")
        else:
            print("🖼️ Found direct files. Use a custom Dataset class.")
else:
    print(f"❌ Path NOT found: {path}")
    print("Checking /content/ to see what is actually there:")
    print(os.listdir("/content/"))

✅ Path exists!
Items inside '/content/dataset': ['Camel', 'Bear', 'Lynx', 'Woodpecker', 'Zebra']
📁 Found subfolders (Classes). Use datasets.ImageFolder(path)


In [17]:
# Print the first 20 file paths to see if they look correct
test_ds = ImageDataset("/content/dataset")
print(test_ds.files[:20])

['/content/dataset/Camel/1431422309330f42.jpg', '/content/dataset/Camel/0215b972cb19e575.jpg', '/content/dataset/Camel/08106ac0bce4f1c0.jpg', '/content/dataset/Camel/408cd6b071fd0546.jpg', '/content/dataset/Camel/962a74b1418c8f14.jpg', '/content/dataset/Camel/e9c80cc9d12c6992.jpg', '/content/dataset/Camel/76f8fac151aa253f.jpg', '/content/dataset/Camel/e18dedb8deb8a34b.jpg', '/content/dataset/Camel/4bc1027f1ccb353d.jpg', '/content/dataset/Camel/b49844caf0ecc14c.jpg', '/content/dataset/Camel/17694cf57865e6b1.jpg', '/content/dataset/Camel/02db256a75c9419a.jpg', '/content/dataset/Camel/f8b69bc9cb1d75ef.jpg', '/content/dataset/Camel/17648cf45f4ab8cd.jpg', '/content/dataset/Camel/d741344d81466170.jpg', '/content/dataset/Camel/0f7062aec3674569.jpg', '/content/dataset/Camel/1c68ec73964a0332.jpg', '/content/dataset/Camel/392f07b11b16ab47.jpg', '/content/dataset/Camel/3a3dabb794c13492.jpg', '/content/dataset/Camel/b591487db3b1bb2a.jpg']


In [8]:
class ImageDataset(Dataset):
    def __init__(self, root, size=64):
        # Fixed the typo: "**/*.jpg" instead of "**/*,jpg"
        self.files = glob(os.path.join(root, "**/*.jpg"), recursive=True)
        
        if len(self.files) == 0:
            print(f"⚠️ Warning: No .jpg files found in {root}. Check path or extensions.")
            
        self.transform = transforms.Compose([
            transforms.Resize(size),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3) # Scales to [-1, 1]
        ])
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        return self.transform(img)

In [9]:
class MappingNetwork(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        layers = []
        for _ in range(8):
            layers.append(nn.Linear(w_dim, w_dim))
            layers.append(nn.LeakyReLU())
        self.mapping = nn.Sequential(*layers)
    
    def forward(self, z):
        z = z / z.norm(dim=1, keepdim=True)
        return self.mapping(z)

In [10]:
class StyleConv(nn.Module):
    def __init__(self, in_ch, out_ch, w_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_ch, in_ch, 3, 3))
        self.style = nn.Linear(w_dim, in_ch)
        self.noise_strength = nn.Parameter(torch.zeros(1))
        self.bias = nn.Parameter(torch.zeros(out_ch))
    
    def forward(self, x, w, noise):
        b, c, h, w_ = x.shape
        style = self.style(w).view(b, 1, c, 1, 1)
        weight = self.weight.unsqueeze(0) * style
        weight = weight.view(-1, c, 3, 3)

        x = x.view(1, -1, h, w_)
        x= F.conv2d(x, weight, padding=1, groups=b)
        x = x.view(b, -1, h, w_)

        x = x + self.noise_strength * noise
        return x + self.bias.view(1, -1, 1, 1)

In [11]:
class Generator(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        self.mapping = MappingNetwork(z_dim, w_dim)

        self.const = nn.Parameter(torch.randn(1, 512, 4, 4))

        self.layers = nn.ModuleList([
            StyleConv(512, 512, w_dim),
            StyleConv(512, 256, w_dim),
            StyleConv(256, 128, w_dim),
            StyleConv(128, 64, w_dim),
          ])
        
        self.to_rgb = nn.Conv2d(64, 3, 1)

    def forward(self, z):
        w = self.mapping(z)
        x = self.const.repeat(z.size(0), 1, 1, 1)

        for layer in self.layers:
            noise = torch.randn_like(x[:, :1])
            x = F.interpolate(x, scale_factor=2, mode='bilinear')
            x = layer(x, w, noise)
            x = F.leaky_relu(x, 0.2)

        return torch.tanh(self.to_rgb(x))

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 4, 2, 1),
                nn.LeakyReLU(0.2)
            )
        
        self.net = nn.Sequential(
            block(3, 64),
            block(64, 128),
            block(128, 256),
            block(256, 512),
            nn.Flatten(),
            nn.Linear(512*4*4, 1)
        )

    def forward(self, x):
        return self.net(x)


In [13]:
def Gradient_Penalty(D, real, fake):
    alpha = torch.rand(real.size(0), 1, 1, 1).to(device)
    interp = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    out = D(interp)

    grads = torch.autograd.grad(
        outputs=out,
        inputs=interp,
        grad_outputs=torch.ones_like(out),
        create_graph=True,
    )[0]

    return ((grads.norm(2, dim=1) - 1) ** 2).mean()

In [14]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    path = "/content/dataset"  # Update this path as needed
    dataset = ImageDataset(path, size=64)
    print(f"✅ Dataset loaded with {len(dataset)} images.")
    loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)

    G = Generator().to(device)
    D = Discriminator().to(device)

    g_opt = torch.optim.Adam(G.parameters(), lr=1e-4, betas=(0.0, 0.99))
    d_opt = torch.optim.Adam(D.parameters(), lr=1e-4, betas=(0.0, 0.99))

    for epoch in range(50):
        for i, real in enumerate(loader):
            real = real.to(device)
            z = torch.randn(real.size(0), 512).to(device)
            fake = G(z)

            d_loss = D(fake).mean() - D(real).mean()
            gp = Gradient_Penalty(D, real, fake)
            d_total = d_loss + 10 * gp

            d_opt.zero_grad()
            d_total.backward()
            d_opt.step()

            if i % 5 == 0:
                g_loss = -D(G(z)).mean()
                g_opt.zero_grad()
                g_loss.backward()
                g_opt.step()

        with torch.no_grad():
            samples = G(torch.randn(16, 512).to(device))
            save_image(samples, f"samples/epoch_{epoch}.png", normalize=True)

        print(f"Epoch {epoch} | D: {d_total.item():.3f} | G: {g_loss.item():.3f}")

In [15]:
if __name__ == "__main__":
    train()

✅ Dataset loaded with 1162840 images.


RuntimeError: The size of tensor a (8) must match the size of tensor b (4) at non-singleton dimension 3